In [2]:
import os
import numpy as np
from skimage.io import imread, imread_collection
from skimage.transform import resize
from sklearn.model_selection import train_test_split

RANDOM_STATE = 69

<h1>Segment images into different classifications</h1>

In [ ]:
#Size to crop images to (Only do, 128x128, 224x224, or 256x256)
img_size = (256, 256)
DATA_PATH = "../data/all"

print("RUNNING")

healthy_images = imread_collection(f"{DATA_PATH}/NORMAL/*.jpeg")

#print("RUNNING 2")

#Crop all images to standard size for training, and add array of labels (0=Normal, 1=Virus, 2=Bacteria)
X = [resize(img, img_size) for img in healthy_images]
Y = [0] * len(X)

index = 0
#Get all sick images, resize, and classify
for file_name in os.listdir(f"{DATA_PATH}/PNEUMONIA"):

    if(index > 1000): #REMOVE LATER#############################################################
        break
    else:
        index += 1
    
    path = os.path.join(f"{DATA_PATH}/PNEUMONIA", file_name)
    img = resize(imread(path), img_size)
    X.append(img)

    if "virus" in file_name:
        Y.append(1)
    elif "bacteria" in file_name:
        Y.append(2)

#Numpy array are very slow at appends, so convert it after modification
X = X.flatten()
Y = Y.flatten()

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=RANDOM_STATE
)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.20, random_state=RANDOM_STATE
)


RUNNING
RUNNING 2


KeyboardInterrupt: 

<h1>Preprocess the data</h1>

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

X_train = X_train.reshape(X.shape[0], img_size[0] * img_size[1])

sm = SMOTE(random_state=RANDOM_STATE)
X_smote, Y_smote = sm.fit_resample(X_train, Y_train)

before_smote = sum(Counter(Y_train).values)
after_smote = sum(Counter(Y_smote).values)

X_smote = X_smote.reshape(after_smote, 256,256)


(1583, 256, 256)